In [9]:
from splinter import Browser
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import pymongo

In [10]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [11]:
# Define database and collection
db = client.jobs_db
db.jobs_glassdoor.drop()
jobs_glassdoor_mongo = db.jobs_glassdoor

In [12]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
job_dict = {}

In [14]:
job_categories = ["Data Analyst", "Data Scientist"]
region = "United States"
posted_time = "7"
for job_category in job_categories:
    
    glassdoor_url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + job_category + \
    "&locT=N&locId=1&locKeyword=" + region + "&jobType=all&fromAge=" + posted_time + \
    "&minSalary=0&includeNoSalaryJobs=true&radius=-1&cityId=-1&minRating=0.0&industryId=-1" +\
    "&companyId=-1&applicationType=0&employerSizes=0&remoteWorkType=0"

    browser.visit(glassdoor_url)
    job_counter = 1
    for x in range(1, 10):
        print("Page no.:", x, "--------")
        
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        job_section = soup.find('ul', class_='jlGrid')
        job_list = job_section.find_all('li', class_='jl')

        for job in job_list:
            # Job Title
            job_title = job.find('div', class_='jobTitle').a.text            
            # Company Name
            company_text = job.find('div', class_='empLoc').text.split("–")
            company_name = company_text[0][1:][:-1] # formatting to remove the spaces in beginning and end
            # City and State
            location = job.find('div', class_='empLoc').span.text.split(", ")          
            # Salary information - Min, Max and Avg
            try:
                salary_text = job.find('span', class_="green").text.replace("(Glassdoor Est.)", "").replace("(Employer Est.)", "")
                sal_range = salary_text[1:].split("-")
                min_sal = int(sal_range[0].lstrip("$").rstrip("k"))
                max_sal = int(sal_range[1].lstrip("$").rstrip("k"))
                avg_sal = (max_sal+min_sal)/2   
            except AttributeError:
                min_sal = None
                max_sal = None
                avg_sal = None                
                
            print('job no.:', job_counter, '-------------')
            job_counter +=1

            job_dict = {"Title": job_title,
                        "Company": company_name,
                        "City": location[0],
                        "State": location[-1],
                        "Designation": job_category,
                        "Min Salary": min_sal,
                        "Max Salary": max_sal,
                        "Avg Salary": avg_sal,
                        "Source": "Glassdoor"
                        }
            jobs_glassdoor_mongo.insert_one(job_dict)
            
        browser.click_link_by_partial_text('Next')
        try:
            #If a pop-up window opens, close    
            browser.find_by_css('.xBtn').click()
        except:
            pass
        sleep(np.random.choice(range(4,7)))
    
    sleep(10)
    

Page no.: 1 --------
job no.: 1 -------------
job no.: 2 -------------
job no.: 3 -------------
job no.: 4 -------------
job no.: 5 -------------
job no.: 6 -------------
job no.: 7 -------------
job no.: 8 -------------
job no.: 9 -------------
job no.: 10 -------------
job no.: 11 -------------
job no.: 12 -------------
job no.: 13 -------------
job no.: 14 -------------
job no.: 15 -------------
job no.: 16 -------------
job no.: 17 -------------
job no.: 18 -------------
job no.: 19 -------------
job no.: 20 -------------
job no.: 21 -------------
job no.: 22 -------------
job no.: 23 -------------
job no.: 24 -------------
job no.: 25 -------------
job no.: 26 -------------
job no.: 27 -------------
job no.: 28 -------------
job no.: 29 -------------
job no.: 30 -------------
job no.: 31 -------------
Page no.: 2 --------
job no.: 32 -------------
job no.: 33 -------------
job no.: 34 -------------
job no.: 35 -------------
job no.: 36 -------------
job no.: 37 -------------
job n

Page no.: 2 --------
job no.: 31 -------------
job no.: 32 -------------
job no.: 33 -------------
job no.: 34 -------------
job no.: 35 -------------
job no.: 36 -------------
job no.: 37 -------------
job no.: 38 -------------
job no.: 39 -------------
job no.: 40 -------------
job no.: 41 -------------
job no.: 42 -------------
job no.: 43 -------------
job no.: 44 -------------
job no.: 45 -------------
job no.: 46 -------------
job no.: 47 -------------
job no.: 48 -------------
job no.: 49 -------------
job no.: 50 -------------
job no.: 51 -------------
job no.: 52 -------------
job no.: 53 -------------
job no.: 54 -------------
job no.: 55 -------------
job no.: 56 -------------
job no.: 57 -------------
job no.: 58 -------------
job no.: 59 -------------
job no.: 60 -------------
job no.: 61 -------------
job no.: 62 -------------
Page no.: 3 --------
job no.: 63 -------------
job no.: 64 -------------
job no.: 65 -------------
job no.: 66 -------------
job no.: 67 ----------

ValueError: invalid literal for int() with base 10: '53\xa0per hour'

In [ ]:
browser.quit()

In [ ]:
jobs_glassdoor_listing = db.jobs_glassdoor.find()
for listing in jobs_glassdoor_listing:
    print(listing)

In [ ]:
scraped_df = pd.DataFrame(list(db.jobs_glassdoor.find()))
scraped_df.head()


In [ ]:
scraped_df.tail()

In [ ]:
cols_list = ['Designation','Company','City','State','Min Salary',\
'Max Salary','Avg Salary','Source']
cols_list
##scraped_df = scraped_df[[4,]]
scraped_df = scraped_df[cols_list]


In [ ]:
scraped_df['Lat'] = ""
scraped_df['Long'] = ""
scraped_df['Title'] = ""

In [ ]:
scraped_df.head()

In [ ]:
file = "Resources/data_jobs.csv"
jobs_flat = pd.read_csv(file, encoding="ISO-8859-1")

In [ ]:
#jobs_flat.head()
jobs_flat.drop('Unnamed: 0',1,inplace = True)
jobs_flat.drop('X',1,inplace = True)

In [ ]:
#create empty columns to push data into
jobs_flat['Min Salary'] = ""
jobs_flat['Max Salary'] = ""
jobs_flat['Avg Salary'] = ""
jobs_flat['Source'] = ""


In [ ]:
jobs_flat = jobs_flat.rename(columns = {
                               "Latitude": "Lat",
                               "Longitude": "Long",
                              })

In [ ]:
#jobs_flat.head()
jobs_flat.drop('reviews',1,inplace = True)
jobs_flat.drop('description',1,inplace = True)


In [ ]:
jobs_flat.head()

In [ ]:
flat_col_list = ['Designation', 'Company', 'City', 'State', 'Min Salary', 'Max Salary',
       'Avg Salary', 'Source', 'Lat', 'Long', 'Title']
jobs_flat = jobs_flat[flat_col_list]

In [ ]:
jobs_flat.head()

In [ ]:
frames = [jobs_flat,scraped_df]

In [ ]:
df_merged = pd.concat(frames)

In [ ]:
df_merged.shape

In [ ]:
jobs_dict = df_merged.to_dict('index')
jobs_dict

In [ ]:
len(jobs_dict)

In [ ]:
db = client.jobs_db
db.job_listings.drop()
jobs_mongo = db.job_listings

In [ ]:
for i in range(len(jobs_dict)):
    jobs_mongo.insert_one(jobs_dict[i])

In [ ]:
listings = db.job_listings.find()
for listing in listings:
    print(listing)

In [ ]:

zips_file = "Resources/uscitiesv.csv"
jobs_flat = pd.read_csv(zips_file)